# Mouse Simulator


This agent represents a very simple mouse who maintains a generative bayesian model of its world consisting of 2 normal distributions and a non-linear transformation g(x) = x^2. The mouse's sensations of its world are very simplistic: it only has a single sensor that generates a scalar value representing light intensity.


In [57]:
import numpy as np

# CookieSensorEnv represents a world that has a cookie and a sensor in it.
# In this world, only one cookie exists at a time, and that cookie
# reflects light as a function of its size.
# An agent senses the light bouncing off the cookie by calling step(),
# which returns a scalar representing light intensity.
# Cookie size is random, determined by a normal distribution with
# with parameters specified via __init__() or left to their default values.
# Sensor output is also random, specified by a 2nd normal distribution,
# with mean set to a function g(x) of the output of the cookie_size
# distribution, and std specified via __init__(). To summarize the flow:
# 0) an agent calls step() on this env, receives a sample of light_intensity
#    generated per the following:
# 1) a cookie is generated with a random size per normal dist. with user
#    provided mean and std.
# 2) size transformed and used as light_mean via: light_mean = g(size) = size^2
# 3) a sensor reading is generated and returned with random intensity per
#    normal distribution using light_mean as mean and user provided std.
class CookieSensorEnv:
    def __init__(self,
                 cookie_size_mean=2,
                 cookie_size_std=1,
                 area_to_light_fn=lambda x: x*x,
                 light_intensity_std=1):
        self.cookie_size_mean = cookie_size_mean
        self.cookie_size_std = cookie_size_std
        self.area_to_light_fn = area_to_light_fn
        self.light_intensity_std = light_intensity_std

    # No actions accepted
    # Returns light intensity at this timestep.
    def step(self, action):
        assert not action
        cookie_size_sample = np.random.normal(self.cookie_size_mean, self.cookie_size_std)
        light_intensity_mean = self.area_to_light_fn(cookie_size_sample)
        light_intensity_sample = np.random.normal(light_intensity_mean, self.light_intensity_std)
        return light_intensity_sample, 0, False, {}

    
# The mouse brain has variables to track its beliefs about the world.
# Beliefs consist of estimates of the parameters in the environment,
# which from the mouses point of view are latent variables that must
# be learned through repeated experience of the end result of the physical
# processes, namely the output of the mouses noisy light sensor (its eye).
#
# These variables fall roughly into two categories:
# 1) beliefs that can be updated quickly in response to sensory input
#    such as the belief that a recently viewed cookie was 2cm.
#    These are analogous to the near-realtime changes to the output
#    of a Tensorflow DNN when new inputs are applied to it.
# 2) beliefs that evolve more slowly over time which in the brain
#    are implemented using synaptic plasticity, analogous to the way
#    params are updated via backprop in TensorFlow.

class Mouse:
    def __init__(self, env, steps=1):
        self.light_intensity_error_belief = 0  # epsilon_u
        self.cookie_size_error_belief = 0      # epsilon_p
        self.cookie_size_belief = 0            # phi
        
        self.cookie_size_std_belief = 1                     # sigma_p
        self.light_intensity_std_belief = 1                 # sigma_u
        self.cookie_size_mean_belief = 3                    # v_p
        self.area_to_light_belief_fn = lambda x: x**2       # g()
        self.area_to_light_deriv_belief_fn = lambda x: 2*x  # g'()
      
        self.step_size = 0.01
        self.env = env
        self.num_steps = 0
        while self.num_steps < steps:
            obs, reward, done, _ = self.env.step(None)
            #print(f"photo receptor returned light intensity reading {obs}")
            self.update_world_model(obs)
            self.num_steps += 1
        
    #paper
    def update_world_model(self, obs):
        def df_dphi(phi):
            # TODO: Replace appropriate terms below with epsilon_u and epsilon_p
            return (self.cookie_size_mean_belief - phi) / self.cookie_size_std_belief + \
                   (obs - phi * phi) / self.light_intensity_std_belief * self.area_to_light_deriv_belief_fn(phi)

        deriv_of_f = df_dphi(self.cookie_size_belief)
        self.cookie_size_belief = self.cookie_size_belief + self.step_size * deriv_of_f
        if self.num_steps % 5 == 0:
            print(f"{self.num_steps}: self.cookie_size_belief is {self.cookie_size_belief}")
        
        
print(f"Random CookieSensorEnv reading: {CookieSensorEnv().step(None)}")
Mouse(CookieSensorEnv(), steps=100)
        

Random CookieSensorEnv reading: (3.0283971217484895, 0, False, {})
0: self.cookie_size_belief is 0.03
5: self.cookie_size_belief is 0.21044409401377204
10: self.cookie_size_belief is 0.41801936556099994
15: self.cookie_size_belief is 0.690422294224483
20: self.cookie_size_belief is 1.0988503349639758
25: self.cookie_size_belief is 1.758162459382944
30: self.cookie_size_belief is 2.515716319237547
35: self.cookie_size_belief is 2.8579689742827776
40: self.cookie_size_belief is 2.3220851707019428
45: self.cookie_size_belief is 2.3899476373454807
50: self.cookie_size_belief is 1.990753296012001
55: self.cookie_size_belief is 2.030191578207341
60: self.cookie_size_belief is 2.604052335678062
65: self.cookie_size_belief is 2.5309756824359306
70: self.cookie_size_belief is 2.3650868568411854
75: self.cookie_size_belief is 1.8639144989056078
80: self.cookie_size_belief is 2.1603188771582884
85: self.cookie_size_belief is 2.187057801649372
90: self.cookie_size_belief is 2.003137643707797
95: s